## Packages and Libraries

In [1]:
import os
import numpy as np
import random
import torch
import torch.nn as nn
from torch import tensor
from torch.nn import Linear, Sequential, ReLU, Identity, BatchNorm1d as BN
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree,remove_self_loops
from torch_geometric.data import Data
from torch_geometric.datasets import TUDataset,Planetoid,GNNBenchmarkDataset
from torch_geometric.loader import DataLoader
from torch_scatter import scatter_mean
from torch.autograd.function import InplaceFunction
from torch_geometric.nn import GCNConv,GINConv,global_mean_pool,TopKPooling
import torch_geometric.transforms as T

from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import time
import argparse
import statistics as stat
from tabulate import tabulate
from collections import OrderedDict

# CPU and Enegusage 
import psutil
import itertools
import tracemalloc
import gc




device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Functions for Mmeasuring criterias

In [2]:
def get_num_parameters(model: nn.Module, count_nonzero_only=False) -> int:
    """
    calculate the total number of parameters of model
    :param count_nonzero_only: only count nonzero weights
    """
    num_counted_elements = 0
    for param in model.parameters():
        if count_nonzero_only:
            num_counted_elements += param.count_nonzero()
        else:
            num_counted_elements += param.numel()
    return num_counted_elements

# Function to get CPU usage
def get_cpu_usage():
    return psutil.cpu_percent(interval=1)



# Function to approximate power consumption (Assume some average power usage per CPU percentage point)
def estimate_power_usage(cpu_usage):
    base_power_usage = 10  # Assumed base power usage in watts
    power_per_percent = 0.5  # Assumed additional watts per CPU usage percent
    return base_power_usage + (power_per_percent * cpu_usage)



def calculate_model_size(model: nn.Module, 
                         qypte: str = 'fp32', 
                         include_metadata: bool = False,
                         model_path: str = None) -> float:
    """
    Calculate model size in KB/MB for different precisions.
    
    Args:
        model: PyTorch model
        precision: 'fp32' (32-bit float) or 'int4' (4-bit integer)
        include_metadata: Whether to include PyTorch metadata in size calculation
        model_path: If provided, will check actual file size on disk
        
    Returns:
        Size in KB (if include_metadata=False) or actual file size (if include_metadata=True)
    """
    # Get total number of parameters
    total_params = sum(p.numel() for p in model.parameters())
    
    # Calculate theoretical size
    if qypte == 'FP32':
        size_bits = total_params * 32
    elif qypte== 'INT4':
        size_bits = total_params * 4
    elif qypte == 'INT8':
        size_bits = total_params * 8    
   
    
    size_bytes = size_bits / 8
    size_kb = size_bytes / 1024
    
    # If checking actual file size
    if include_metadata and model_path:
        if not os.path.exists(model_path):
            # Save model to temporary file if path doesn't exist
            torch.save(model.state_dict(), model_path)
        actual_size_kb = os.path.getsize(model_path) / 1024
        return actual_size_kb
    
    return size_kb




Byte = 8
KiB = 1024 * Byte
MiB = 1024 * KiB
GiB = 1024 * MiB

## GIN Model

In [3]:
## GIN Model

import torch
from torch_geometric.nn import GINConv

class GIN(nn.Module):
    def __init__(self, dataset, num_layers, hidden_units, num_deg=1000, init='norm'):
        super(GIN, self).__init__()
        self.conv1 = GINConv(
            nn.Sequential(
                nn.Linear(9, hidden_units),
                nn.ReLU(),
                nn.Linear(hidden_units, hidden_units),
                nn.ReLU(),
                nn.BatchNorm1d(hidden_units),
            ),
            train_eps=True,
        )
        self.convs = torch.nn.ModuleList()
        
        for _ in range(num_layers - 1):
            self.convs.append(
                GINConv(
                    nn.Sequential(
                        nn.Linear(hidden_units, hidden_units),
                        nn.ReLU(),
                        nn.Linear(hidden_units, hidden_units),
                        nn.ReLU(),
                        nn.BatchNorm1d(hidden_units),
                    ),
                    train_eps=True,
                )
            )
        self.bn_list = torch.nn.ModuleList()
        for _ in range(num_layers):
            self.bn_list.append(nn.BatchNorm1d(hidden_units))
        
        self.lin1 = nn.Linear(hidden_units, hidden_units)
        self.lin2 = nn.Linear(hidden_units, dataset.num_classes)

    def reset_parameters(self):
        self.conv1.reset_parameters()
        for conv in self.convs:
            conv.reset_parameters()
        self.lin1.reset_parameters()
        self.lin2.reset_parameters()

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        #print(self.conv1(x, edge_index))
       # print(self.conv1(x, edge_index).size)
        x = self.conv1(x, edge_index)
        x = self.bn_list[0](x)
        i = 1
        for conv in self.convs:
            x= conv(x, edge_index)
            x = self.bn_list[i](x)
            i += 1
        x = global_mean_pool(x, batch)
        x = self.lin1(x)
        x = F.relu(x)
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)
    




# Helpful Function

In [4]:


class NormalizedDegree(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, data):
        deg = degree(data.edge_index[0], dtype=torch.float)
        deg = (deg - self.mean) / self.std
        data.x = deg.view(-1, 1)
        return data


def num_graphs(data):
    if data.batch is not None:
        return data.num_graphs
    else:
        return data.x.size(0)


def train(model, optimizer, loader,a_loss, a_storage=1):
    model.train()
    total_loss = 0
    for data in loader:
        optimizer.zero_grad()
        data = data.to(device)
        #out,bit_sum = model(data)
        out = model(data)
        target = data.y.view(-1)

        # Check if the last batch is smaller
        if out.size(0) != target.size(0):
            target = target[:out.size(0)]  # Truncate target to match output size

        loss = F.cross_entropy(out, target)
        #loss_store = a_loss*F.relu(bit_sum-a_storage)**2
        #loss_store.backward(retain_graph=True)
        loss.backward()
        total_loss += loss.item() * num_graphs(data)
        optimizer.step()
    return total_loss / len(loader.dataset)

##



def eval_acc(model, loader):
    model.eval()
    correct = 0
    for data in loader:        
        data = data.to(device)
        with torch.no_grad():
            #pred = model(data)[0].max(1)[1] 
            pred = model(data).max(1)[1]

        target = data.y.view(-1)
        # Check if the last batch is smaller
        if pred.size(0) != target.size(0):
            target = target[:pred.size(0)]  # Truncate target to match output size                 
        correct+=pred.eq(target).sum().item()
    return correct / len(loader.dataset)


def eval_loss(model, loader):
    model.eval()

    loss = 0
    for data in loader:     
        data = data.to(device)
        with torch.no_grad():
           #out = model(data)[0]
            out = model(data)

        target = data.y.view(-1)

        # Check if the last batch is smaller
        if out.size(0) != target.size(0):
            target = target[:out.size(0)]  # Truncate target to match output size                
        loss += F.cross_entropy(out, target, reduction="sum").item()
     
    return loss / len(loader.dataset)


# Real k_fold
def k_fold(dataset, folds):
    skf = StratifiedKFold(folds, shuffle=True, random_state=12345)

    test_indices, train_indices = [], []
    for _, idx in skf.split(torch.zeros(len(dataset)), dataset.data.y):
        test_indices.append(torch.from_numpy(idx))

    val_indices = [test_indices[i - 1] for i in range(folds)]

    for i in range(folds):
        train_mask = torch.ones(len(dataset), dtype=torch.bool)
        train_mask[test_indices[i]] = 0
        train_mask[val_indices[i]] = 0
        train_indices.append(train_mask.nonzero().view(-1))

    return train_indices, test_indices, val_indices


def load_checkpoint(model, checkpoint):
    if checkpoint != 'No':
        print("loading checkpoint...")
        model_dict = model.state_dict()
        modelCheckpoint = torch.load(checkpoint)
        pretrained_dict = modelCheckpoint['state_dict']
        new_dict = {k: v for k, v in pretrained_dict.items() if ((k in model_dict.keys()))}
        model_dict.update(new_dict)
        print('Total : {}, update: {}'.format(len(pretrained_dict), len(new_dict)))
        model.load_state_dict(model_dict)
        print("loaded finished!")
    return model

## Setting Argument

In [5]:
import sys
import argparse

# Clearing the arguments
sys.argv = ['']


parser = argparse.ArgumentParser()
parser.add_argument('--model',type=str,default='GIN')
parser.add_argument('--gpu_id',type=int,default=0)
parser.add_argument('--dataset_name',type=str,default='BBBP')
parser.add_argument('--num_deg',type=int,default=1000)
parser.add_argument('--num_layers', type=int, default=5)
parser.add_argument('--hidden_units',type=int,default=64)
parser.add_argument('--batch-size',type=int,default=64)
parser.add_argument('--bit',type=int,default=4)
parser.add_argument('--max_epoch',type=int,default=100)
parser.add_argument('--max_cycle',type=int,default=2000)
parser.add_argument('--folds',type=int,default=10)
parser.add_argument('--weight_decay',type=float,default=0)
parser.add_argument('--lr',type=float,default=0.01)
parser.add_argument('--a_loss',type=float,default=0.001)
parser.add_argument('--lr_quant_scale_fea',type=float,default=0.02)
parser.add_argument('--lr_quant_scale_xw',type=float,default=1e-2)
parser.add_argument('--lr_quant_scale_weight',type=float,default=0.02)
parser.add_argument('--lr_quant_bit_fea',type=float,default=0.008)
parser.add_argument('--lr_quant_bit_weight',type=float,default=0.0001)
parser.add_argument('--lr_step_size',type=int, default=50)
parser.add_argument('--lr_decay_factor',type=float,default=0.5)
parser.add_argument('--lr_schedule_patience',type=int,default=10)
parser.add_argument('--is_naive',type=bool,default=False)
###############################################################
parser.add_argument('--resume',type=bool,default=True)
parser.add_argument('--store_ckpt',type=bool,default=True)
parser.add_argument('--uniform',type=bool,default=True)
parser.add_argument('--use_norm_quant',type=bool,default=True)
###############################################################
# The target memory size of nodes features
parser.add_argument('--a_storage',type=float,default=1)
# Path to results
parser.add_argument('--result_folder',type=str,default='result')
# Path to checkpoint
parser.add_argument('--check_folder',type=str,default='checkpoint')
# Path to dataset
parser.add_argument('--pathdataset',type=str,default='/')

args = parser.parse_args()
print(args)

Namespace(model='GIN', gpu_id=0, dataset_name='BBBP', num_deg=1000, num_layers=5, hidden_units=64, batch_size=64, bit=4, max_epoch=100, max_cycle=2000, folds=10, weight_decay=0, lr=0.01, a_loss=0.001, lr_quant_scale_fea=0.02, lr_quant_scale_xw=0.01, lr_quant_scale_weight=0.02, lr_quant_bit_fea=0.008, lr_quant_bit_weight=0.0001, lr_step_size=50, lr_decay_factor=0.5, lr_schedule_patience=10, is_naive=False, resume=True, store_ckpt=True, uniform=True, use_norm_quant=True, a_storage=1, result_folder='result', check_folder='checkpoint', pathdataset='/')


In [6]:
###############################################################
model = args.model
dataset_name = args.dataset_name
num_layers = args.num_layers
hidden_units=args.hidden_units
bit=args.bit
max_epoch = args.max_epoch
resume = args.resume


# Path direction
pathresult = args.result_folder+'/'+args.model+'_'+dataset_name
pathcheck = args.check_folder+'/'+args.model+'_'+dataset_name
if not os.path.exists(pathresult):
    os.makedirs(pathresult)
if not os.path.exists(pathcheck):
    os.makedirs(pathcheck)
###############################################################

## Loading Dataset and Normalization

In [7]:
from torch_geometric.datasets import MoleculeNet
def load_MolecueNet(dataset_dir, dataset_name, task=None):
    """ Attention the multi-task problems not solved yet """
    molecule_net_dataset_names = {name.lower(): name for name in MoleculeNet.names.keys()}
    dataset = MoleculeNet(root=dataset_dir, name=molecule_net_dataset_names[dataset_name.lower()])
    dataset.data.x = dataset.data.x.float()
    if task is None:
        dataset.data.y = dataset.data.y.squeeze().long()
    else:
        dataset.data.y = dataset.data.y[task].long()
    dataset.node_type_dict = None
    dataset.node_color = None
    return dataset

def get_dataset(dataset_dir, dataset_name, task=None):
   
    molecule_net_dataset_names = [name.lower() for name in MoleculeNet.names.keys()]
    dataset=load_MolecueNet(dataset_dir, dataset_name, task)

    return  dataset
    


def get_dataloader(dataset, batch_size, random_split_flag=True, data_split_ratio=None, seed=2):
    """
    Args:
        dataset:
        batch_size: int
        random_split_flag: bool
        data_split_ratio: list, training, validation and testing ratio
        seed: random seed to split the dataset randomly
    Returns:
        a dictionary of training, validation, and testing dataLoader
    """

    if not random_split_flag and hasattr(dataset, 'supplement'):
        assert 'split_indices' in dataset.supplement.keys(), "split idx"
        split_indices = dataset.supplement['split_indices']
        train_indices = torch.where(split_indices == 0)[0].numpy().tolist()
        dev_indices = torch.where(split_indices == 1)[0].numpy().tolist()
        test_indices = torch.where(split_indices == 2)[0].numpy().tolist()

        train = Subset(dataset, train_indices)
        eval = Subset(dataset, dev_indices)
        test = Subset(dataset, test_indices)
    else:
        num_train = int(0.8 * len(dataset))
        num_eval = int(0.1 * len(dataset))
        num_test = len(dataset) - num_train - num_eval

        train, eval, test = random_split(dataset, lengths=[num_train, num_eval, num_test],
                                         generator=torch.Generator().manual_seed(seed))

    dataloader = dict()
    dataloader['train'] = DataLoader(train, batch_size=batch_size, shuffle=True)
    dataloader['eval'] = DataLoader(eval, batch_size=batch_size, shuffle=False)
    dataloader['test'] = DataLoader(test, batch_size=batch_size, shuffle=False)
    return dataloader


In [8]:

args.dataset_name=='BBBP'
dataset = get_dataset(args.pathdataset, args.dataset_name)

Processing...
C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\torch_geometric\datasets\molecule_net.py:213: UserWarning: Skipping molecule 'O=N([O-])C1=C(CN=C1NCCSCc2ncccc2)Cc3ccccc3' since it resulted in zero atoms
  warnings.warn(f"Skipping molecule '{smiles}' since it "
C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\torch_geometric\datasets\molecule_net.py:213: UserWarning: Skipping molecule 'c1(nc(NC(N)=[NH2])sc1)CSCCNC(=[NH]C#N)NC' since it resulted in zero atoms
  warnings.warn(f"Skipping molecule '{smiles}' since it "
C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\torch_geometric\datasets\molecule_net.py:213: UserWarning: Skipping molecule 'Cc1nc(sc1)\[NH]=C(\N)N' since it resulted in zero atoms
  warnings.warn(f"Skipping molecule '{smiles}' since it "
C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\torch_geometric\datasets\molecule_net.py:213: UserWarning: Skipping molecule 's1cc(C

In [9]:

if dataset.data.x is None:
    max_degree = 0
    degs = []
    for data in dataset:
        degs += [degree(data.edge_index[0], dtype=torch.long)]
        max_degree = max(max_degree, degs[-1].max().item())

    if max_degree < 1000:
        dataset.transform = T.OneHotDegree(max_degree)
    else:
        deg = torch.cat(degs, dim=0).to(torch.float)
        mean, std = deg.mean().item(), deg.std().item()
        dataset.transform = NormalizedDegree(mean, std)

In [10]:
def run(bit=32, max_epoch=5):
        args.batch_size=16
        args.max_epoch=5
        args.batch_size=32
        max_acc =0.65
        if bit== 32:
            qypte='FP32'
        elif bit== 4:
            qypte= 'INT4'    
        elif bit== 8:
            qypte = 'INT8'
            
        
     
      
        val_losses, accu, durations = [], [], []
        main_model_accuracy=[]
        main_model_loss=[]
        t_main_model=[]
        Num_parm_main_model=[]
        main_model_size=[]
        main_energy_consumption=[]
        main_cpu_usage=[]
        main_memory_usage=[]
       
 
        #Eva= OrderedDict()
        #Eva=dict()
        # Initialize a dictionary to store all results per iteration
        Eva_iter = {
            "val losses per iter": [],
            "durations per iter": [],
            "main model accuracy per iter": [],
            "time inference of main model per iter": [],
            "number parmameters of main model per iter": [],  # Store the best accuracy for each fold
            "size of main model per iter": [],
            "energy consumption of main model per iter": [],
            "cpu usage of main model per iter": [],
            "total memory usage of main model per iter": [],
            "final_metrics": {}  # Store final metrics (mean, std, etc.)
        }
    
        
     


        for fold, (train_idx, test_idx, val_idx) in enumerate(zip(*k_fold(dataset, args.folds))):
            print_max_acc=0
            train_dataset = dataset[train_idx.tolist()]
            test_dataset = dataset[test_idx.tolist()]
            val_dataset = dataset[val_idx.tolist()]
            train_loader = DataLoader(train_dataset, args.batch_size, num_workers=0,shuffle=False, drop_last=True)
            val_loader = DataLoader(val_dataset, args.batch_size,num_workers=0,shuffle=False,drop_last=True)
            test_loader = DataLoader(test_dataset, args.batch_size,num_workers=0,shuffle=False, drop_last=True)
            k=0


            model=GIN(train_dataset, args.num_layers,hidden_units=args.hidden_units,
                num_deg=args.num_deg).to(device)
            optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=args.lr_step_size, gamma=args.lr_decay_factor)
            
            t_start = time.perf_counter()

            Eva_fold= OrderedDict() #It is a dictionary to arrange output of this fold
        
            for epoch in range(max_epoch):
                #t = tqdm(epoch)
                train_loss=0
                train_loss = train(model,optimizer,train_loader,args.a_loss, args.a_storage)
                start = time.process_time()
                val_loss = eval_loss(model,val_loader)
                val_losses.append(val_loss)
                end = time.process_time()
                acc = eval_acc(model,test_loader)
                

                if epoch % 30 == 0:
                    print(f"Eval Epoch: {epoch} |Val_loss:{val_loss:.03f}| Train_Loss: {train_loss:.3f} | Acc: {acc:.3f}|Fold: {fold}")
                accu.append(acc)
                if(acc>max_acc):
                    max_acc = acc
                    #path=pathcheck+'/'+args.model+'_'+dataset_name+'_'+str(bit)+'bit'+'main.pth.tar'
                    main_model_path='main.pth'
                    torch.save({'state_dict': model.state_dict()}, main_model_path)
                if(acc>print_max_acc):
                    print_max_acc = acc

            t_end = time.perf_counter()
            durations.append(t_end - t_start)      
                    
            # Start monitoring CPU and memory usage, model size, number of parametes, time inference and  power consumption
            #main_model_path=pathcheck+'/'+args.model+'_'+dataset_name+'_'+str(bit)+'bit'+'main.pth.tar'
            main_model_path='main.pth'
            #state = torch.load(main_model_path)
            #dict=state['state_dict']
            #recover_model = lambda: model.load_state_dict(state['state_dict'])

            gc.collect()
            time.sleep(5)  # Add a 5-second delay to stabilize the initial state
            tracemalloc.start()  # Start tracking memory allocations
            snapshot_before = tracemalloc.take_snapshot()#take a snapshot of the current memory state before starting the measurement.

            t0 = time.perf_counter()
            initial_cpu_usage = get_cpu_usage()
            power_usage = estimate_power_usage(initial_cpu_usage)


            fold_main_model_accuracy= eval_acc(model, test_loader)

            fold_main_cpu_usage = get_cpu_usage()
            t1 = time.perf_counter()
            fold_t_main_model=t1-t0

            snapshot_after = tracemalloc.take_snapshot()
            tracemalloc.stop()
            top_stats = snapshot_after.compare_to(snapshot_before, 'lineno')

            folde_main_total_memory_diff = sum([stat.size_diff for stat in top_stats])
            fold_main_energy_consumption = power_usage * fold_t_main_model
            fold_main_model_size = os.path.getsize(main_model_path)
            #fold_main_model_size =calculate_model_size(model, qypte )
            fold_num_parm_main_model=get_num_parameters(model, count_nonzero_only=True)
            

            gc.collect()
            time.sleep(5) 
            #Update Eva dictionary
            Eva_fold.update({'main model accuracy per fold': fold_main_model_accuracy,
                        'time inference of main model per fold':fold_t_main_model,
                        'number parmameters of main model per fold': fold_num_parm_main_model,
                        'size of main model per fold': fold_main_model_size, 
                        'energy consumption of main model per fold':fold_main_energy_consumption,
                        'total memory usage of main model per fold':folde_main_total_memory_diff,
                        'cpu usage of main model per fold':fold_main_cpu_usage
                       })

            gc.collect()
            time.sleep(5) 


            main_model_accuracy.append(Eva_fold['main model accuracy per fold'])
            t_main_model.append(Eva_fold['time inference of main model per fold'])
            Num_parm_main_model.append(int(Eva_fold['number parmameters of main model per fold']))
            main_model_size.append(int(Eva_fold['size of main model per fold']))
            main_energy_consumption.append(Eva_fold['energy consumption of main model per fold'])
            main_cpu_usage.append(Eva_fold['cpu usage of main model per fold'])
            main_memory_usage.append(Eva_fold['total memory usage of main model per fold'])





        Eva_iter["main model accuracy per iter"]= stat.mean(main_model_accuracy)
        Eva_iter["time inference of main model per iter"]= stat.mean(t_main_model)
        Eva_iter["number parmameters of main model per iter"]=  stat.mean(Num_parm_main_model)
        Eva_iter["size of main model per iter"]= stat.mean(main_model_size)
        Eva_iter["energy consumption of main model per iter"]= stat.mean(main_energy_consumption)
        Eva_iter["cpu usage of main model per iter"]= stat.mean(main_cpu_usage)
        Eva_iter["total memory usage of main model per iter"]= stat.mean(main_memory_usage)


        loss, acc, duration = tensor(val_losses), tensor(accu), tensor(durations)
        loss, acc = loss.view(args.folds, max_epoch), acc.view(args.folds, max_epoch)
        loss, argmin = loss.min(dim=1)
        acc = acc[torch.arange(args.folds, dtype=torch.long), argmin]

        Eva_iter["val losses per iter"]= loss.mean().item()
        Eva_iter["durations per iter"]= duration.mean().item()




        return Eva_iter , model                                       


## Manual Measurement

In [11]:
# The following are all list of criteria for measurements. 
# We collect all desired datas of each list across iterations. 
# Then, we compute average and std of each list.



#Main model
Main_val_loss=[]
Main_duration=[]
Main_model_accuracy=[]
T_Main_model=[]
Num_parm_Main_model=[]
Main_model_size=[]
Main_Energy_Consumption=[]
Main_Cpu_Usage=[]
Main_Memory_Usage=[]


# Here is the dictionary to record the list of all measurements
Eva_measure={'Main validation loss':Main_val_loss,
             'Main duration':Main_duration,
            'Main model accuracy': Main_model_accuracy,
            'time inference of Main model':T_Main_model,
            'number parmameters of Main model':Num_parm_Main_model,
            'Main model size':Main_model_size,
            'energy consumption of Main model':Main_Energy_Consumption,
            'cpu usage of Main model':Main_Cpu_Usage,
            'memory usage of Main model':Main_Memory_Usage}


In [14]:
args.max_epoch=100
max_epoch = args.max_epoch
iterations=4
args.bit=32
bit=args.bit
folds=10

In [15]:
#### load the Mainized  model

for i in range(iterations):
    print('********************************************')
    print(f'The iteration is :{i+1} ')
   
 

    
    Eva_iter, model=run(bit, max_epoch)

 
    Main_val_loss.append(Eva_iter["val losses per iter"])
    Main_duration.append(Eva_iter["durations per iter"])
    Main_model_accuracy.append(Eva_iter["main model accuracy per iter"])
    T_Main_model.append(Eva_iter["time inference of main model per iter"])
    Num_parm_Main_model.append(Eva_iter["number parmameters of main model per iter"])
    Main_model_size.append(Eva_iter["size of main model per iter"])
    Main_Energy_Consumption.append(Eva_iter["energy consumption of main model per iter"])
    Main_Cpu_Usage.append( Eva_iter["cpu usage of main model per iter"])
    Main_Memory_Usage.append(Eva_iter["total memory usage of main model per iter"])
 

********************************************
The iteration is :1 


C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\torch_geometric\data\in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


Eval Epoch: 0 |Val_loss:2.509| Train_Loss: 0.518 | Acc: 0.706|Fold: 0
Eval Epoch: 30 |Val_loss:0.683| Train_Loss: 0.549 | Acc: 0.706|Fold: 0
Eval Epoch: 60 |Val_loss:8.445| Train_Loss: 0.555 | Acc: 0.598|Fold: 0
Eval Epoch: 90 |Val_loss:0.527| Train_Loss: 0.560 | Acc: 0.706|Fold: 0
Eval Epoch: 0 |Val_loss:8.450| Train_Loss: 0.508 | Acc: 0.706|Fold: 1
Eval Epoch: 30 |Val_loss:1.552| Train_Loss: 0.542 | Acc: 0.412|Fold: 1
Eval Epoch: 60 |Val_loss:808037.744| Train_Loss: 0.553 | Acc: 0.672|Fold: 1
Eval Epoch: 90 |Val_loss:1135981.980| Train_Loss: 0.553 | Acc: 0.672|Fold: 1
Eval Epoch: 0 |Val_loss:4.576| Train_Loss: 0.489 | Acc: 0.706|Fold: 2
Eval Epoch: 30 |Val_loss:711.379| Train_Loss: 0.551 | Acc: 0.706|Fold: 2
Eval Epoch: 60 |Val_loss:0.543| Train_Loss: 0.553 | Acc: 0.706|Fold: 2
Eval Epoch: 90 |Val_loss:464.267| Train_Loss: 0.553 | Acc: 0.706|Fold: 2
Eval Epoch: 0 |Val_loss:7.385| Train_Loss: 0.516 | Acc: 0.706|Fold: 3
Eval Epoch: 30 |Val_loss:0.529| Train_Loss: 0.553 | Acc: 0.706|Fol

C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\torch_geometric\data\in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


Eval Epoch: 0 |Val_loss:3.091| Train_Loss: 0.526 | Acc: 0.706|Fold: 0
Eval Epoch: 30 |Val_loss:1.057| Train_Loss: 0.568 | Acc: 0.706|Fold: 0
Eval Epoch: 60 |Val_loss:4.618| Train_Loss: 0.431 | Acc: 0.706|Fold: 0
Eval Epoch: 90 |Val_loss:3.502| Train_Loss: 0.559 | Acc: 0.706|Fold: 0
Eval Epoch: 0 |Val_loss:2.564| Train_Loss: 0.504 | Acc: 0.706|Fold: 1
Eval Epoch: 30 |Val_loss:25477.706| Train_Loss: 0.514 | Acc: 0.706|Fold: 1
Eval Epoch: 60 |Val_loss:782.211| Train_Loss: 0.548 | Acc: 0.706|Fold: 1
Eval Epoch: 90 |Val_loss:6.133| Train_Loss: 0.548 | Acc: 0.706|Fold: 1
Eval Epoch: 0 |Val_loss:3.344| Train_Loss: 0.508 | Acc: 0.706|Fold: 2
Eval Epoch: 30 |Val_loss:1629790607127090.250| Train_Loss: 0.655 | Acc: 0.706|Fold: 2
Eval Epoch: 60 |Val_loss:98.779| Train_Loss: 0.553 | Acc: 0.706|Fold: 2
Eval Epoch: 90 |Val_loss:98.779| Train_Loss: 0.553 | Acc: 0.706|Fold: 2
Eval Epoch: 0 |Val_loss:3.785| Train_Loss: 0.486 | Acc: 0.706|Fold: 3
Eval Epoch: 30 |Val_loss:0.542| Train_Loss: 0.591 | Acc: 0

C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\torch_geometric\data\in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


Eval Epoch: 0 |Val_loss:3.999| Train_Loss: 0.503 | Acc: 0.706|Fold: 0
Eval Epoch: 30 |Val_loss:4755210.877| Train_Loss: 0.553 | Acc: 0.235|Fold: 0
Eval Epoch: 60 |Val_loss:0.571| Train_Loss: 0.555 | Acc: 0.706|Fold: 0
Eval Epoch: 90 |Val_loss:0.526| Train_Loss: 0.556 | Acc: 0.706|Fold: 0
Eval Epoch: 0 |Val_loss:3.299| Train_Loss: 0.504 | Acc: 0.706|Fold: 1
Eval Epoch: 30 |Val_loss:0.532| Train_Loss: 0.565 | Acc: 0.706|Fold: 1
Eval Epoch: 60 |Val_loss:2178387.961| Train_Loss: 0.546 | Acc: 0.250|Fold: 1
Eval Epoch: 90 |Val_loss:0.571| Train_Loss: 0.553 | Acc: 0.701|Fold: 1
Eval Epoch: 0 |Val_loss:2.226| Train_Loss: 0.530 | Acc: 0.706|Fold: 2
Eval Epoch: 30 |Val_loss:154.430| Train_Loss: 0.557 | Acc: 0.250|Fold: 2
Eval Epoch: 60 |Val_loss:0.989| Train_Loss: 0.553 | Acc: 0.686|Fold: 2
Eval Epoch: 90 |Val_loss:0.989| Train_Loss: 0.553 | Acc: 0.686|Fold: 2
Eval Epoch: 0 |Val_loss:3.362| Train_Loss: 0.516 | Acc: 0.706|Fold: 3
Eval Epoch: 30 |Val_loss:0.530| Train_Loss: 0.553 | Acc: 0.706|Fold

C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\torch_geometric\data\in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


Eval Epoch: 0 |Val_loss:2.545| Train_Loss: 0.502 | Acc: 0.706|Fold: 0
Eval Epoch: 30 |Val_loss:34.888| Train_Loss: 0.673 | Acc: 0.230|Fold: 0
Eval Epoch: 60 |Val_loss:0.527| Train_Loss: 0.560 | Acc: 0.706|Fold: 0
Eval Epoch: 90 |Val_loss:0.527| Train_Loss: 0.560 | Acc: 0.706|Fold: 0
Eval Epoch: 0 |Val_loss:3.729| Train_Loss: 0.510 | Acc: 0.706|Fold: 1
Eval Epoch: 30 |Val_loss:3397930648242744320.000| Train_Loss: 0.548 | Acc: 0.706|Fold: 1
Eval Epoch: 60 |Val_loss:0.530| Train_Loss: 0.553 | Acc: 0.706|Fold: 1
Eval Epoch: 90 |Val_loss:0.530| Train_Loss: 0.553 | Acc: 0.706|Fold: 1
Eval Epoch: 0 |Val_loss:2.761| Train_Loss: 0.506 | Acc: 0.706|Fold: 2
Eval Epoch: 30 |Val_loss:46792213.333| Train_Loss: 0.558 | Acc: 0.706|Fold: 2
Eval Epoch: 60 |Val_loss:0.530| Train_Loss: 0.553 | Acc: 0.706|Fold: 2
Eval Epoch: 90 |Val_loss:0.530| Train_Loss: 0.553 | Acc: 0.706|Fold: 2
Eval Epoch: 0 |Val_loss:3.364| Train_Loss: 0.490 | Acc: 0.706|Fold: 3
Eval Epoch: 30 |Val_loss:0.530| Train_Loss: 0.553 | Acc

In [17]:
Eva_iter["main model accuracy per iter"]

0.7042717086834734

In [18]:
Main_model_accuracy

[0.6807422969187675,
 0.6978991596638655,
 0.5876050420168067,
 0.7057422969187676,
 0.7042717086834734]

In [19]:
[0.6807422969187675,
 0.6978991596638655,
 0.5876050420168067,
 0.7057422969187676,
 0.7042717086834734]

[0.6807422969187675,
 0.6978991596638655,
 0.5876050420168067,
 0.7057422969187676,
 0.7042717086834734]

In [16]:
# This is a dictionary to save all measurements. Aftre measuring, we can compute mean and std of each item.
from collections import OrderedDict 
Eva_final = OrderedDict()

print(f"All measurement about Main model of type:{ bit} ")   

main_model_val_loss_mean =stat.mean(Main_val_loss)
main_model_val_loss_std = stat.stdev(Main_val_loss)
main_model_val_loss = "{:.3f} ± {:.3f}".format(main_model_val_loss_mean,main_model_val_loss_std)
print("main_model_val_loss - {}".format(main_model_val_loss))


Eva_final.update({'Ave of main loss validation':float(format(main_model_val_loss_mean, '.3f'))})
Eva_final.update({'Std of main loss validation':float(format(main_model_val_loss_std, '.3f'))}) 

###########

main_model_duration_mean =stat.mean(Main_duration)
main_model_duration_std = stat.stdev(Main_duration)
main_model_duration = "{:.3f} ± {:.3f}".format(main_model_duration_mean,main_model_duration_std )
print("main model duration - {}".format(main_model_duration))

Eva_final.update({'Ave of main model duration':float(format(main_model_duration_mean , '.3f'))})
Eva_final.update({'Std of main model duration':float(format(main_model_duration_std, '.3f'))})                                         
                                     
#############
main_model_accuracy_mean =stat.mean(Main_model_accuracy)
main_model_accuracy_std = stat.stdev(Main_model_accuracy)
main_model_accuracy = "{:.3f} ± {:.3f}".format(main_model_accuracy_mean,main_model_accuracy_std )
print("quant model accuracy - {}".format(main_model_accuracy))

Eva_final.update({'Ave of main model accuracy':float(format(main_model_accuracy_mean, '.3f'))})
Eva_final.update({'Std of main model accuracy':float(format(main_model_accuracy_std, '.3f'))})
                 
######################
t_main_model_mean = stat.mean(T_Main_model)
t_main_model_std =stat.stdev(T_Main_model)
t_main_model = "{:.3f} ± {:.3f}".format(main_model_accuracy_mean,main_model_accuracy_std )
print("time inference of main model- {}".format(t_main_model))
      
Eva_final.update({'Ave of time inference of main model':float(format(t_main_model_mean, '.3f'))})
Eva_final.update({'Std of time inference of main model':float(format(t_main_model_std, '.3f'))})

##############
num_parm_main_model_mean = stat.mean(Num_parm_Main_model)
num_parm_main_model_std = stat.stdev(Num_parm_Main_model)
num_parm_main_model= "{:.3f} ± {:.3f}".format(num_parm_main_model_mean, num_parm_main_model_std)
print("num parm main model: {}".format(num_parm_main_model))


Eva_final.update({'Ave of number parmameters of main model':num_parm_main_model_mean})
Eva_final.update({'Std of number parmameters of main model':num_parm_main_model_std})
##########################
main_model_size_mean =stat.mean( Main_model_size)
main_model_size_std = stat.stdev(Main_model_size)
main_model_size= "{:.3f} ± {:.3f}".format(main_model_size_mean, main_model_size_std)
print("main modelsize: {}".format(main_model_size))


Eva_final.update({'Ave of main model size':main_model_size_mean})
Eva_final.update({'Std of main_model_size':main_model_size_std })

###########################
main_energy_consumption_mean = stat.mean(Main_Energy_Consumption)
main_energy_consumption_std = stat.stdev(Main_Energy_Consumption)
main_energy_consumption= "{:.3f} ± {:.3f}".format(main_energy_consumption_mean, main_energy_consumption_std)
print("main energy consumption: {}".format(main_energy_consumption))


Eva_final.update({'Ave of energy consumption of main model':main_energy_consumption_mean })
Eva_final.update({'Std of energy consumption of main model':main_energy_consumption_std})

###############################
main_cpu_usage_mean = stat.mean(Main_Cpu_Usage)
main_cpu_usage_std = stat.stdev(Main_Cpu_Usage)
main_cpu_usage= "{:.3f} ± {:.3f}".format(main_cpu_usage_mean, main_cpu_usage_std )
print("main cpu usage: {}".format(main_cpu_usage))

Eva_final.update({'Ave of cpu usage of main model':main_cpu_usage_mean})
Eva_final.update({'Std of cpu usage of main model':main_cpu_usage_std})


###############
main_memory_usage_mean = stat.mean(Main_Memory_Usage)
main_memory_usage_std = stat.stdev(Main_Memory_Usage)
main_memory_usage= "{:.3f} ± {:.3f}".format(main_memory_usage_mean, main_memory_usage_std)
print("main memory usage: {}".format(main_memory_usage))

Eva_final.update({'Ave of memory usage of main model':main_memory_usage_mean})
Eva_final.update({'Std of memory usage of main model':main_memory_usage_std})

#################################


# Determing Mainization Method 




All measurement about Main model of type:32 
main_model_val_loss - 0.519 ± 0.002
main model duration - 418.954 ± 112.761
quant model accuracy - 0.675 ± 0.050
time inference of main model- 0.675 ± 0.050
num parm main model: 43655.000 ± 0.000
main modelsize: 199749.000 ± 0.000
main energy consumption: 73.138 ± 46.588
main cpu usage: 39.410 ± 27.563
main memory usage: 45707.060 ± 199.006


In [17]:
# Record the accuracy

mode='FP32'
file_name = pathresult+'/'+args.model+'_'+dataset_name+'_'+str(bit)+'bit'+'_with_Mode_'+mode+'.txt'

with open(file_name, 'w') as f:
    for key, value in vars(args).items():
        f.write('%s:%s\n'%(key, value))

    for key, value in Eva_final.items():
        f.write('%s:%s\n'%(key, value))

    for key, value in Eva_measure.items():
        f.write('%s:%s\n' % (key, ','.join(map(str, value))))  